# Rijksmuseum MCP Demo

This notebook demonstrates how to use the MCP (Machine Callable Programs) integration with the Rijksmuseum API. It allows you to query information about artworks, search for specific pieces, and even open images in your browser.

## Setup

First, let's import the necessary libraries and set up our MCP server connection.

In [11]:
import os
import nest_asyncio
import asyncio
import getpass
from haystack import Pipeline
from haystack.components.converters import OutputAdapter
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.tools import ToolInvoker
from haystack.dataclasses import ChatMessage

from haystack_integrations.tools.mcp.connection import MCPServer, StdioServerInfo


nest_asyncio.apply()

## Configure API Keys

You'll need to set your Rijksmuseum API key and OpenAI API key. You can get a Rijksmuseum API key by registering at [Rijksmuseum API](https://www.rijksmuseum.nl/en/research/conduct-research/data).

In [12]:
# Set your API keys here using getpass for security
RIJKSMUSEUM_API_KEY = getpass.getpass("Enter your Rijksmuseum API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Path to your MCP server implementation
NODE_PATH = "node"  # Update this to your Node.js path
MCP_SERVER_PATH = "/rijksmuseum-mcp/build/index.js"  # Update this to your MCP server path

Enter your Rijksmuseum API key:  ········
Enter your OpenAI API key:  ········


## Initialize MCP Server

Now let's initialize the MCP server connection and explore the available tools.

In [13]:
# Initialize the MCP server connection
try:
    server = MCPServer(
        server_info=StdioServerInfo(
            command=NODE_PATH, 
            args=[MCP_SERVER_PATH], 
            env={"RIJKSMUSEUM_API_KEY": RIJKSMUSEUM_API_KEY}
        ),
        connection_timeout=30  # Increase timeout if needed
    )
    
    print("\nAvailable tools on server:")
    tools = server.available_tools
    for tool in tools:
        print(f"\nTool: {tool.name}")
        print(f"Description: {tool.description}")
        print("Parameters:", tool.parameters)
        print("-" * 50)
except Exception as e:
    print(f"Error initializing MCP server: {e}")
    print("\nTroubleshooting tips:")
    print("1. Make sure your Node.js path is correct")
    print("2. Make sure your MCP server path is correct")
    print("3. Make sure your Rijksmuseum API key is valid")
    print("4. Try running the server manually to check for errors")
    raise


Available tools on server:

Tool: search_artwork
Description: Search for artworks in the Rijksmuseum collection
Parameters: {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search terms to find artwork (e.g. title, artist, etc)'}, 'pageSize': {'type': 'number', 'description': 'Number of results to return (1-100)', 'minimum': 1, 'maximum': 100, 'default': 10}}, 'required': ['query']}
--------------------------------------------------

Tool: get_artwork_details
Description: Get detailed information about a specific artwork
Parameters: {'type': 'object', 'properties': {'objectNumber': {'type': 'string', 'description': 'The identifier of the artwork (e.g. SK-C-5 for The Night Watch)'}}, 'required': ['objectNumber']}
--------------------------------------------------

Tool: get_artwork_image
Description: Get image tiles information for an artwork
Parameters: {'type': 'object', 'properties': {'objectNumber': {'type': 'string', 'description': 'The identifier of t

## Create Pipeline

Let's create a pipeline that uses the MCP server to select and use the appropriate tool for a given query.

In [14]:
# Create pipeline components with all available tools
pipeline = Pipeline()
pipeline.add_component(
    "llm", 
    OpenAIChatGenerator(
        model="gpt-4o", 
        tools=tools,
    )
)
pipeline.add_component(
    "tool_invoker", 
    ToolInvoker(tools=tools)
)
pipeline.add_component(
    "adapter",
    OutputAdapter(
        template="{{ initial_msg + initial_tool_messages + tool_messages }}",
        output_type=list[ChatMessage],
        unsafe=True,
    ),
)
pipeline.add_component(
    "response_llm", 
    OpenAIChatGenerator(model="gpt-4o")
)

# Connect pipeline components
pipeline.connect("llm.replies", "tool_invoker.messages")
pipeline.connect("llm.replies", "adapter.initial_tool_messages")
pipeline.connect("tool_invoker.tool_messages", "adapter.tool_messages")
pipeline.connect("adapter.output", "response_llm.messages")

🚅 Components
  - llm: OpenAIChatGenerator
  - tool_invoker: ToolInvoker
  - adapter: OutputAdapter
  - response_llm: OpenAIChatGenerator
🛤️ Connections
  - llm.replies -> tool_invoker.messages (List[ChatMessage])
  - llm.replies -> adapter.initial_tool_messages (List[ChatMessage])
  - tool_invoker.tool_messages -> adapter.tool_messages (List[ChatMessage])
  - adapter.output -> response_llm.messages (list[ChatMessage])

## Interactive Demo

Now let's create an interactive function to query the pipeline.

In [15]:
def query_rijksmuseum(user_input):
    user_input_msg = ChatMessage.from_user(text=user_input)
    
    try:
        result = pipeline.run({
            "llm": {"messages": [user_input_msg]}, 
            "adapter": {"initial_msg": [user_input_msg]}
        }, include_outputs_from={"tool_invoker"})
        
        print("\nUser:", user_input)
        print("\nAssistant:", result["response_llm"]["replies"][0].text)
        print("\nTool invocations:")
        for tool_call in result["tool_invoker"].get("tool_calls", []):
            print(f"  - Tool: {tool_call.get('name')}")
            print(f"    Arguments: {tool_call.get('arguments')}")
            print(f"    Response: {tool_call.get('response')}")
        
        return result
    except Exception as e:
        print(f"Error during query: {e}")

## Example Queries

Let's try some example queries to demonstrate the functionality.

In [ ]:
# Example 1: Search for popular artworks
query_rijksmuseum("What are the most popular artworks in the Rijksmuseum?")

In [ ]:
# Example 2: Get information about Rembrandt
query_rijksmuseum("What years was Rembrandt active?")

In [16]:
# Example 3: Open an image in the browser
query_rijksmuseum("Can you show me The Night Watch in my browser?")


User: Can you show me The Night Watch in my browser?

Assistant: I've opened "The Night Watch" by Rembrandt in your browser. You can view it by following [this link](https://www.rijksmuseum.nl/en/collection/SK-C-5).

Tool invocations:


{'tool_invoker': {'tool_messages': [ChatMessage(_role=<ChatRole.TOOL: 'tool'>, _content=[ToolCallResult(result='Successfully opened image in browser: https://www.rijksmuseum.nl/en/collection/SK-C-5', origin=ToolCall(tool_name='open_image_in_browser', arguments={'imageUrl': 'https://www.rijksmuseum.nl/en/collection/SK-C-5'}, id='call_32cQcvdLEZu5DK9J5ojgDysk'), error=False)], _name=None, _meta={})]},
 'response_llm': {'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='I\'ve opened "The Night Watch" by Rembrandt in your browser. You can view it by following [this link](https://www.rijksmuseum.nl/en/collection/SK-C-5).')], _name=None, _meta={'model': 'gpt-4o-2024-08-06', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 41, 'prompt_tokens': 80, 'total_tokens': 121, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_de

In [ ]:
# Example 4: Get details about a specific artwork
query_rijksmuseum("Get me the details of the artwork called The Night Watch")

## Interactive User Input

Now you can try your own queries!

In [ ]:
from ipywidgets import widgets
from IPython.display import display, HTML

text_input = widgets.Text(
    value='',
    placeholder='Enter your question about Rijksmuseum artworks',
    description='Query:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', 
    tooltip='Submit your query',
    icon='check'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        if text_input.value:
            query_rijksmuseum(text_input.value)
        else:
            print("Please enter a query.")

button.on_click(on_button_clicked)

display(widgets.HBox([text_input, button]))
display(output)

## Example Queries You Can Try

Here are some example queries you can try:

1. "What are the most famous paintings by Vermeer?"
2. "Show me paintings with flowers from the 17th century"
3. "Can you open Girl with a Pearl Earring in my browser?"
4. "Tell me about the history of The Night Watch"
5. "What are some notable Dutch Golden Age paintings in the collection?"
6. "Find artworks depicting Amsterdam in winter"
7. "Show me the details of Rembrandt's self-portraits"
8. "What's the most valuable artwork in the Rijksmuseum?"

## Cleanup

When you're done, make sure to close the MCP server connection.

In [ ]:
# Close the server connection when done
try:
    server.close()
    print("MCP server connection closed successfully.")
except Exception as e:
    print(f"Error closing MCP server connection: {e}")